In [1]:
!pip install powerlaw==1.4.6

In [144]:
from scipy.optimize import curve_fit
from sklearn import preprocessing
import networkx as nx
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.probability import FreqDist
import itertools
import collections
import powerlaw
from nltk import bigrams
from nltk.util import ngrams
import os
import glob
import plotly.express as px
import plotly.graph_objs as go
from nltk.corpus import stopwords
import string
import math
from sklearn.model_selection import train_test_split
# use glob to get all the csv files in the folder



In [106]:
df = pd.read_csv('df_tutti_1.csv')
df.shape

(53581, 30)

# Text preparation

In [107]:
import string
punctuation=string.punctuation.replace('+', '')
def remove_punct(text):
    text  = "".join([char for char in text if char not in punctuation])
    return text

df['Tweet_punct'] = df['renderedContent'].apply(lambda x: remove_punct(x) )

df['Tweet_punct']

0        SonoJu29ro bizzarripaolo giuno55 Mov5Stelle Ha...
1        MichaelPioggia La Francia non ha nessun motivo...
2        SonoJu29ro bizzarripaolo giuno55 Mov5Stelle Pr...
3        AndreaRomano9 pdnetwork PiuEuropa Tra mpo sara...
4        ViVilaVitaOra Non una ma 1000 volte siiiiiiiii...
                               ...                        
53576    tg3  A linea notte sta venendo fuori la ipocri...
53577    PRIMA PAGINA IL MATTINO  MeloniSalvini prove d...
53578    agnesepini “Più che plausibile che Salvini par...
53579    La trincea di Salvini E Giorgetti avverte io p...
53580    No ma credo che nel caso sarebbe lo stesso Mat...
Name: Tweet_punct, Length: 53581, dtype: object

In [108]:
# words_in_tweet
words_in_tweet = [tweet.lower().split() for tweet in df['Tweet_punct']]

In [157]:
#Removing stop-word
stop_words = set(stopwords.words('italian'))

# Remove stop words from each tweet list of words
tweets_nsw = [[word for word in tweet_words if not word in stop_words]
              for tweet_words in words_in_tweet]

#create a unique list from all tweets, and set to dataframe
word= [item for sublist in tweets_nsw for item in sublist]
words=pd.DataFrame()
words['word']=word
words['count'] = df.apply(lambda _: '', axis=1)
tweets_nsw

[['sonoju29ro',
  'bizzarripaolo',
  'giuno55',
  'mov5stelle',
  'chiuso',
  'rubinetti',
  'gas',
  'tutta',
  'europa',
  'cosa',
  'dovuto',
  'fare',
  'ricordiamoci',
  'pure',
  'fin',
  '2014',
  'ucraina',
  'donbas',
  'stati',
  'uccisi',
  'migliaia',
  'oppositori',
  'truppe',
  'nazifasciste',
  'ucraine',
  'nessuno',
  'parla',
  'armi',
  'risolve',
  'pace'],
 ['michaelpioggia',
  'francia',
  'nessun',
  'motivo',
  'fare',
  'parcheggi',
  'italia',
  'russia',
  'là',
  'te',
  'sembra',
  'normale',
  'comportamento',
  'delleuropa',
  'questione'],
 ['sonoju29ro',
  'bizzarripaolo',
  'giuno55',
  'mov5stelle',
  'premetto',
  'difendo',
  'putin',
  'detesto',
  'scatenato',
  'guerra',
  'fratricida',
  'ucraini',
  'russi',
  'condanna',
  'certami',
  'chiedo',
  'tutta',
  'europa',
  'schiera',
  'ucraina',
  'giustocon',
  'intenzione',
  'impoverirla',
  'mentre',
  'russia',
  'putin',
  'ritorsione',
  'cosa',
  'fatto'],
 ['andrearomano9', 'pdnetwork'

In [158]:
#Create a ranking for the words
frequ=FreqDist(word)
rank=pd.DataFrame(frequ.items(), columns=['Word', 'Count'])
rank=rank.set_index('Word')
rank.sort_values(by=['Count'])

,Count
Word,
rinominato,1
didalema,1
deil,1
verdibonelli,1
impegnocivicoluigidivano,1
...,...
calenda,5652
renzi,5925
letta,6135


In [110]:
df['renderedContent']=tweets_nsw

# Create user and mentioned user lists

In [112]:
user=[]
for i in range(0, len(df['user'])):
    c=df['user'][i].split("'")
    user.append(c[3])

In [113]:
use = np.array(user) 
unique_user = np.unique(use)
len(unique_user)

22102

In [114]:
men_user=[[] for _ in range(0,len(df['mentionedUsers']))]

for i in range(0,len(df['mentionedUsers'])):
    if isinstance(df['mentionedUsers'][i],float):
        men_user[i].append('nan')
    else:
        v=df['mentionedUsers'][i].split("{")
        v.remove(v[0])
        if len(v)>1:
            for j in range(0, len(v)-1):
                d=v[j].split("'")
                men_user[i].append(d[3])
        elif len(v)==1:
            d=v[0].split("'")
            men_user[i].append(d[3])
    
    

In [115]:
df.drop(df.columns.difference(['renderedContent']), axis=1, inplace=True)

In [116]:
df['user']=user
df['men_user']=men_user

# Build the Network

In [125]:
# Create network plot
G = nx.Graph()

# Create connections between nodes
for k ,v ,m in zip(df['user'].items(),df['men_user'].items(),df['renderedContent'].items()):
    for i in range(0, len(v[1])):
        if v[1][i]=='nan':pass
        else:  
            G.add_edge(k[1], v[1][i], tweet=m[1])        

(G.number_of_nodes(),G.number_of_edges())

(26429, 49998)

In [ ]:
G.remove_edges_from(nx.selfloop_edges(G))
(G.number_of_nodes(),G.number_of_edges(),nx.density(G))

In [ ]:
nx.write_adjlist(G, "elect_dict.edgelist")